# HyLoRADA Experiments

This notebook runs HyLoRADA training, evaluation, and benchmarks on Kaggle.

**Scripts:**
- `train_hylorada.py` - Train HyLoRADA model
- `evaluate_hylorada.py` - Evaluate trained model
- `run_benchmark.py` - Compare all PEFT methods

In [ ]:
# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['GRPC_VERBOSITY'] = 'ERROR'

In [ ]:
# Clone repo
!git clone https://github.com/SadiaTabassum1216/hylorada.git
%cd hylorada

In [ ]:
!pip install -q transformers datasets accelerate tqdm

In [ ]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

## Option 1: Train HyLoRADA Only

In [ ]:
# Train HyLoRADA model
!python train_hylorada.py \
    --model Qwen/Qwen2-0.5B \
    --max_length 1024 \
    --epochs 3 \
    --batch_size 1 \
    --grad_accum 16 \
    --num_samples 1000 \
    --output_dir ./output

## Option 2: Evaluate Trained Model

In [ ]:
# Evaluate trained HyLoRADA model
!python evaluate_hylorada.py \
    --model Qwen/Qwen2-0.5B \
    --weights ./output/final/hylorada_weights.pt \
    --max_length 1024 \
    --num_samples 100

## Option 3: Run Full Benchmark (All Methods)

In [ ]:
# Compare all PEFT methods with same hyperparameters
!python run_benchmark.py \
    --model Qwen/Qwen2-0.5B \
    --max_length 1024 \
    --epochs 1 \
    --batch_size 1 \
    --grad_accum 16 \
    --num_train 1000 \
    --num_test 100 \
    --lora_rank 8 \
    --output_dir ./benchmark_results

In [ ]:
# Compare specific methods only
# !python run_benchmark.py --model Qwen/Qwen2-0.5B --epochs 3 --methods lora lorada hylorada